## Solutions for Unit 4 Actions Part 2

<img src="../img/robotignite_logo_text.png"/>

## Index: 

* <a href="#SolutionExercise4-13">Solution Exercise 4.13</a>
* <a href="#SolutionExercise4-14">Solution Exercise 4.14</a>

## Solution Exercise 4.13 <p id="SolutionExercise4-13"></p>

<p style="background:#EE9023;color:white;">**Exercise 4.13**</p>

For this exercise, we will assume that our package is called **exercise_413**, our launch file is called **move_drone_square.launch**, and our C++ file is called **move_drone_square.cpp**.

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: move_drone_square.launch** </p>

In [ ]:
<launch>
    <node pkg="exercise_413" type="move_drone_square" name="move_drone_square" output="screen" />
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: move_drone_square.launch** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**C++ File: move_drone_square.cpp** </p>

In [ ]:
#include <ros/ros.h>
#include <actionlib/server/simple_action_server.h>
#include <actionlib/TestAction.h>
#include <geometry_msgs/Twist.h>
#include <std_msgs/Empty.h>

class MoveSquareAction
{
protected:

  ros::NodeHandle nh_;
  // NodeHandle instance must be created before this line. Otherwise strange error occurs.
  actionlib::SimpleActionServer<actionlib::TestAction> as_; 
  std::string action_name_;
  // create messages that are used to publish feedback and result
  actionlib::TestFeedback feedback_;
  actionlib::TestResult result_;
  
  // Create needed messages
  int rate_hz_;
  bool success_;
  int sideSeconds_;
  int turnSeconds_;
  
  ros::Rate *rate_;      
  ros::Publisher move_pub_;
  geometry_msgs::Twist move_msg_;
  ros::Publisher takeoff_pub_;
  std_msgs::Empty takeoff_msg_;
  ros::Publisher land_pub_;
  std_msgs::Empty land_msg_;

public:

  MoveSquareAction(std::string name) :
    as_(nh_, name, boost::bind(&MoveSquareAction::executeCB, this, _1), false),
    action_name_(name)
  {
    as_.start();
    rate_hz_ = 1;
    success_ = true;
    turnSeconds_ = 2;
    
    rate_ = new ros::Rate(rate_hz_);
    move_pub_ = nh_.advertise<geometry_msgs::Twist>("/cmd_vel", 1000);
    takeoff_pub_ = nh_.advertise<std_msgs::Empty>("/drone/takeoff", 1000);
    land_pub_ = nh_.advertise<std_msgs::Empty>("/drone/land", 1000);
  }

  ~MoveSquareAction(void)
  {
  }

  void executeCB(const actionlib::TestGoalConstPtr &goal)
  {
    
    // Takeoff the drone
    this->takeoff_drone();

    // start executing the action
    for(int i=0; i<4; i++)
    {
      // check that preempt has not been requested by the client
      if (as_.isPreemptRequested() || !ros::ok())
      {
        ROS_INFO("%s: Preempted", action_name_.c_str());
        // set the action state to preempted
        as_.setPreempted();
        success_ = false;
        break;
      }
      
      //Perdorm the square
      sideSeconds_ = goal->goal;
      this->move_forward_drone(sideSeconds_);
      this->turn_drone(turnSeconds_);
      
            
      feedback_.feedback = i;
      // publish the feedback
      as_.publishFeedback(feedback_);
      // this sleep is not necessary, the sequence is computed at 1 Hz for demonstration purposes
      rate_->sleep();
    }

    if(success_)
    {
      result_.result = (sideSeconds_*4) + (turnSeconds_*4);
      ROS_INFO("The total seconds it took the drone to perform the square was %i seconds", result_.result);
      ROS_INFO("%s: Succeeded", action_name_.c_str());
      // set the action state to succeeded
      as_.setSucceeded(result_);
      // Stop and land drone
      this->stop_drone();
      this->land_drone();
    }
  }
    
  // Functions to control the drone    
  void stop_drone(void)
  {
    ROS_INFO("Stopping Drone...");
    int i = 0;
    while (i < 3)
    {
      move_msg_.linear.x = 0;
      move_msg_.angular.z = 0;
      move_pub_.publish(move_msg_);
      i++;
      rate_->sleep();
    }
  }
    
  void move_forward_drone(int side_secs)
  {
    ROS_INFO("Moving forward Drone...");
    int i = 0;
    while (i < side_secs)
    {
      move_msg_.linear.x = 1;
      move_msg_.angular.z = 0;
      move_pub_.publish(move_msg_);
      i++;
      rate_->sleep();
    }
  }
    
  void turn_drone(int turn_secs)
  {
    ROS_INFO("Turning Drone...");
    int i = 0;
    while (i < turn_secs)
    {
      move_msg_.linear.x = 0;
      move_msg_.angular.z = 0.55;
      move_pub_.publish(move_msg_);
      i++;
      rate_->sleep();
    }
  }
  
  void takeoff_drone(void)
  {
    ROS_INFO("Taking Off Drone...");
    int i = 0;
    while (i < 4)
    {
      takeoff_pub_.publish(takeoff_msg_);
      i++;
      rate_->sleep();
    }
  }
  
  void land_drone(void)
  {
    ROS_INFO("Landing Drone...");
    int i = 0;
    while (i < 4)
    {
      land_pub_.publish(land_msg_);
      i++;
      rate_->sleep();
    }
  }

};

int main(int argc, char** argv)
{
  ros::init(argc, argv, "move_square_drone");

  MoveSquareAction move_square("move_drone_square_as");
  ros::spin();

  return 0;
}

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END C++ File: move_drone_square.cpp** </p>

As you can see, in the above code we are using the seconds to move in each side of the square in order to define if the square will be bigger or smaller. So, for more seconds defined in the goal, the square will be bigger. This could be done in many ways, like checking the odometry to identify the distance the drone is actually moving, but that's quite more complex, and it's not the purpose of the exercise. The purpose of this Exercise is that you learn how to build a proper Action Server, and interact with the goal, feedback and result of the Action.

In order to test this code, you have to first launch the Action Server (which is defined in the above C++ file). You can do that by using the following command:

In [ ]:
roslaunch exercise_413 move_drone_saquare.launch

Now, if you execute the following command:

In [ ]:
rostopic list | grep move_drone_square

You will see something like this:

<img src="../img/move_drone_square_as.png" width="600" />

This means you Action Server is up and ready to receive a goal.

So now, let's publish a goal to this Action Server. You can do this by two methods (as you saw in the Part 1 of the Actions notebooks):

* Creating an Action Client
* Publishing a goal directly through the action topics

For this case, let's use the 2nd method. You should then publish a message to the **move_drone_square_as/goal** topic, just like this:

<img src="../img/goal_square_sol.png" width="800" />

You will see the drone doing something like this:

<img src="../img/drone_square_sol.gif" width="600" />

Finally, you can check the **feedback** and **result** topics of the Action, to check if they're publishing the desired values.

In [ ]:
rostopic echo /move_drone_square_as/feedback

<img src="../img/feedback_square_sol.png" width="600" />

In [ ]:
rostopic echo /move_drone_square_as/result

<img src="../img/result_square_sol.png" width="600" />

## Solution Exercise 4.14 <p id="SolutionExercise4-14"></p>

<p style="background:#EE9023;color:white;">**Exercise 4.14**</p>

For this exercise, we will assume that our package is called **exercise_414**, our launch file is called **action_custom_msg.launch**, and our C++ file is called **action_custom_msg.cpp**.

So, for the exercise 4.14, we will have to create a custom action message that will defines the movement of the drone by using a string, that can be **UP** or **DOWN**. As feedback, it will also return a string indicating which action is taking place at the moment. As a result, it will return nothing.

For that, the first you will have to do is to create a folder called **action** inside the **exercise_414** package.

In [ ]:
roscd exercise_414
mkdir action

Then, inside this **action** folder, you will have to create a file called **CustomActionMsg.action**, with the following content inside it:

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Message File: CustomActionMsg.action** </p>

In [ ]:
string goal
---
---
string feedback

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Message File: CustomActionMsg.action** </p>

As you can see, we define both the goal and the feedback as strings, while we leave the result in blank.

Then, you will also have to modify the **CMakeLists.txt** and **package.xml** files, as it is described in the Actions Notebooks. If you are lost and don't know how to proceed, below you can check working examples of this files:

<p style="background:#3B8F10;color:white;" id="prg-2-1">**CMakeLists.txt** </p>

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(exercise_414)

find_package(catkin REQUIRED COMPONENTS
  actionlib
  actionlib_msgs
  roscpp
)

## Generate actions in the 'action' folder
add_action_files(
   FILES
   CustomActionMsg.action
#   Action2.action
 )

generate_messages(
   DEPENDENCIES
   actionlib_msgs
 )

catkin_package(
  CATKIN_DEPENDS actionlib actionlib_msgs roscpp
)

###########
## Build ##
###########

add_executable(action_custom_msg src/action_custom_msg.cpp)
add_dependencies(action_custom_msg ${action_custom_msg_EXPORTED_TARGETS} ${catkin_EXPORTED_TARGETS})
target_link_libraries(action_custom_msg
${catkin_LIBRARIES}
)

add_dependencies(action_custom_msg exercise_414_generate_messages_cpp)

include_directories(
  ${catkin_INCLUDE_DIRS}
)


<p style="background:#3B8F10;color:white;" id="prg-2-1">**END CMakeLists.txt** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**package.xml** </p>

In [ ]:
<?xml version="1.0"?>
<package>
  <name>exercise_414</name>
  <version>0.0.0</version>
  <description>The my_custom_action_msg_pkg package</description>
  <maintainer email="user@todo.todo">user</maintainer>
  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>actionlib</build_depend>
  <build_depend>actionlib_msgs</build_depend>
  <build_depend>roscpp</build_depend>
  <run_depend>actionlib</run_depend>
  <run_depend>actionlib_msgs</run_depend>
  <run_depend>roscpp</run_depend>

  <export>
  </export>
</package>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END package.xml** </p>

Once all of these is done, you will need to compile your package and source **ALL the webwhells** that you are going to use so that ROS can find the new Messages.

roscd
cd ..
catkin_make
source devel/setup.bash

And finally, check if you can find your new message:

In [ ]:
rosmsg list | grep CustomActionMsg

You should see something like this:

In [ ]:
exercise_414/CustomActionMsgAction
exercise_414/CustomActionMsgActionFeedback
exercise_414/CustomActionMsgActionGoal
exercise_414/CustomActionMsgActionResult
exercise_414/CustomActionMsgFeedback
exercise_414/CustomActionMsgGoal
exercise_414/CustomActionMsgResult

Once this is done and working, you can proceed to create the Action Server that will use this new message. Below you can check both the launch file and the C++ file:

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: action_custom_msg.launch** </p>

In [ ]:
<launch>
    <node pkg="exercise_414" type="action_custom_msg" name="action_custom_msg" output="screen" />
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: action_custom_msg.launch** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**C++ File: action_custom_msg.cpp** </p>

In [ ]:
#include <ros/ros.h>
#include <actionlib/server/simple_action_server.h>
#include <exercise_414/CustomActionMsgAction.h>
#include <geometry_msgs/Twist.h>
#include <std_msgs/Empty.h>

class MoveSquareAction
{
protected:

  ros::NodeHandle nh_;
  // NodeHandle instance must be created before this line. Otherwise strange error occurs.
  actionlib::SimpleActionServer<exercise_414::CustomActionMsgAction> as_; 
  std::string action_name_;
  // create messages that are used to publish feedback and result
  exercise_414::CustomActionMsgFeedback feedback_;
  exercise_414::CustomActionMsgResult result_;
  
  // Create needed messages
  int rate_hz_;
  bool success_;
  int sideSeconds_;
  int turnSeconds_;
  std::string upDown_;
  
  ros::Rate *rate_;      
  ros::Publisher move_pub_;
  geometry_msgs::Twist move_msg_;
  ros::Publisher takeoff_pub_;
  std_msgs::Empty takeoff_msg_;
  ros::Publisher land_pub_;
  std_msgs::Empty land_msg_;

public:

  MoveSquareAction(std::string name) :
    as_(nh_, name, boost::bind(&MoveSquareAction::executeCB, this, _1), false),
    action_name_(name)
  {
    as_.start();
    rate_hz_ = 1;
    success_ = true;
    turnSeconds_ = 2;
    
    rate_ = new ros::Rate(rate_hz_);
    move_pub_ = nh_.advertise<geometry_msgs::Twist>("/cmd_vel", 1000);
    takeoff_pub_ = nh_.advertise<std_msgs::Empty>("/drone/takeoff", 1000);
    land_pub_ = nh_.advertise<std_msgs::Empty>("/drone/land", 1000);
  }

  ~MoveSquareAction(void)
  {
  }

  void executeCB(const exercise_414::CustomActionMsgGoalConstPtr &goal)
  {
    
    std::string upDown = goal->goal;

    // start executing the action
    for(int i=0; i<4; i++)
    {
      // check that preempt has not been requested by the client
      if (as_.isPreemptRequested() || !ros::ok())
      {
        ROS_INFO("%s: Preempted", action_name_.c_str());
        // set the action state to preempted
        as_.setPreempted();
        success_ = false;
        break;
      }
      
      if (upDown == "UP")
      {
          this->takeoff_drone();
      }
      
      if (upDown == "DOWN")
      {
          this->land_drone();
      }
      
      feedback_.feedback = i;
      // publish the feedback
      as_.publishFeedback(feedback_);
      
      rate_->sleep();
    }

    if(success_)
    {
      
      ROS_INFO("%s: Succeeded", action_name_.c_str());
      // set the action state to succeeded
      as_.setSucceeded(result_);
      
    }
  }
    
  // Functions to control the drone    
  void stop_drone(void)
  {
    ROS_INFO("Stopping Drone...");
    int i = 0;
    while (i < 3)
    {
      move_msg_.linear.x = 0;
      move_msg_.angular.z = 0;
      move_pub_.publish(move_msg_);
      i++;
      rate_->sleep();
    }
  }
    
  void move_forward_drone(int side_secs)
  {
    ROS_INFO("Moving forward Drone...");
    int i = 0;
    while (i < side_secs)
    {
      move_msg_.linear.x = 1;
      move_msg_.angular.z = 0;
      move_pub_.publish(move_msg_);
      i++;
      rate_->sleep();
    }
  }
    
  void turn_drone(int turn_secs)
  {
    ROS_INFO("Turning Drone...");
    int i = 0;
    while (i < turn_secs)
    {
      move_msg_.linear.x = 0;
      move_msg_.angular.z = 0.55;
      move_pub_.publish(move_msg_);
      i++;
      rate_->sleep();
    }
  }
  
  void takeoff_drone(void)
  {
    ROS_INFO("Taking Off Drone...");
    int i = 0;
    while (i < 4)
    {
      takeoff_pub_.publish(takeoff_msg_);
      i++;
      rate_->sleep();
    }
  }
  
  void land_drone(void)
  {
    ROS_INFO("Landing Drone...");
    int i = 0;
    while (i < 4)
    {
      land_pub_.publish(land_msg_);
      i++;
      rate_->sleep();
    }
  }

};

int main(int argc, char** argv)
{
  ros::init(argc, argv, "action_custom_msg");

  MoveSquareAction move_square("action_custom_msg_as");
  ros::spin();

  return 0;
}

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END C++ File: action_custom_msg.cpp** </p>

As you can see, in the above code we are using the **/drone/takeoff** and **/drone/land** topics in order to move UP or DOWN, since they make the drone move aproximately 1 meter. You could also use the **/cmd_vel** topic, but that's up to you.

In order to test this code, you have to first launch the Action Server (which is defined in the above C++ file). You can do that by using the following command:

In [ ]:
roslaunch exercise_414 action_custom_msg.launch

Now, if you execute the following command:

In [ ]:
rostopic list | grep action_custom_msg

You will see something like this:

<img src="../img/action_custom_msg_as.png" width="600" />

This means you Action Server is up and ready to receive a goal.

So now, let's publish a goal to this Action Server. You can do this by two methods (as you saw in the Part 1 of the Actions notebooks):

* Creating an Action Client
* Publishing a goal directly through the action topics

For this case, let's use the 2nd method. You should then publish a message to the **action_custom_msg_as/goal** topic, just like this:

<img src="../img/action_custom_msg_as_goal.png" width="800" />

You will see the drone doing something like this:

<img src="../img/up_or_down1.gif" width="600" />

<img src="../img/up_or_down2.gif" width="600" />

Finally, you can check the **feedback** and **result** topics of the Action, to check if they're publishing the desired values.

In [ ]:
rostopic echo /action_custom_msg_as/feedback

<img src="../img/action_custom_msg_as_feedback.png" width="600" />

<img src="../img/action_custom_msg_as_feedback2.png" width="600" />

In this case, there's no result to be shown.